In [122]:
from langchain_openai import ChatOpenAI
import pandas as pd
import os
from typing import List
from enum import Enum

from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain.output_parsers.enum import EnumOutputParser


with open('config/config.txt') as f:
    for line in f:
        env_data = line.split(',')
        os.environ[env_data[0]] = env_data[1]

llm = ChatOpenAI(model_name='gpt-3.5-turbo',temperature=0.7)

reviews = pd.read_csv('data/review_sample.csv', index_col=0)

In [126]:
def extract_significant_ideas(text):
    # Define the prompt to instruct the model to extract significant ideas
    prompt = f"Please summarize the following text by extracting the significant ideas as short sentences. SEPARATE EACH SENTENCE WITH COMMAS ONLY:\n\n{text}"
    
    # Use LangChain's llm to call the OpenAI API
    response = llm.invoke(prompt)

    return response

ideas = {}

# Extract significant ideas
for i in range(5):
    significant_ideas = extract_significant_ideas(reviews.text[i])
    ideas[reviews.review_id[i]] = significant_ideas.content.split(',')

In [127]:
ideas

{'QEkyAdsrVo7tEsANDN7O9g': ['Missed Marvin',
  ' enjoyed the chocolate shake',
  ' went later in the day than usual',
  ' best time to go is when they open',
  ' Marvin is entertaining.'],
 'kIViMGM4JTHHozAE1r9boA': ["Best breakfast I've had in a very long time",
  ' very disappointed with breakfast',
  " maybe it's just me",
  " Camellia's was amazing",
  ' had the omelet at the bottom of the menu',
  ' so good',
  ' brother had the Mexican which was tasty',
  ' had a pecan omelet that is to die for',
  ' thinner than most',
  ' enjoyed it',
  ' get your order in asap',
  ' service is personable and awesome.'],
 'LpHt30LaQHAtBs-j_EYUUw': ['Visited new French Quarter location',
  ' enjoyed it as much as Carrollton location',
  ' Manhattan Omelette is a must-try',
  ' delicious.'],
 'WqCjrz_fV-n6y_nDxme2lA': ['One of my favorite places',
  ' great food',
  ' high energy service from Marvin',
  ' best breakfast place in the world',
  ' worth the wait',
  ' New Orleans landmark',
  ' bett

In [158]:
class Boolean(Enum):
    TRUE = "TRUE"
    FALSE = "FALSE"

def are_related(idea_0, idea_1):
    parser = EnumOutputParser(enum=Boolean)
    prompt = PromptTemplate.from_template(
        """
        Please look at the following two ideas and decide whether they express the same or a very similar thing. Keep in mind that both ideas are related to a specific restauraunt: 
        
        Idea 1: {idea_0}
        
        Idea 2: {idea_1}

        Instructions: {instructions}
        """
    ).partial(instructions=parser.get_format_instructions())
    chain = prompt | llm | parser
    response = chain.invoke({"idea_0": idea_0, "idea_1": idea_1})
    return response.value == 'TRUE'

def merge_ideas(idea_0, idea_1):
    prompt = f'Given the following two related ideas come up with a few words to summarize them: \n\nIdea 1: {idea_0}\n\nIdea 2: {idea_1}'
    return llm.invoke(prompt).content


a = ideas['kIViMGM4JTHHozAE1r9boA'][5]
b = ideas['LpHt30LaQHAtBs-j_EYUUw'][3]


if are_related(a, b):
    merged = merge_ideas(a, b)
    print(merged)    

Tasty, satisfying, enjoyable


In [157]:
r

True

In [160]:
reviews[reviews.review_id == 'kIViMGM4JTHHozAE1r9boA'].text[1]

'best breakfast ive had in a very long time. ive been very disappointed with breakfast, maybe its just me. thats why cammellias was so amazing i had the omelet on the bottom of the menu maybe camellia, chef  or something. so good! my brother had the mexican which was also very tasty. then we had a pecan omelet and that my friends is to die for. its thinner than most which i really enjoyed, get you order in asap they get busy. the service however is very personable and awesome.'

In [161]:
ideas['kIViMGM4JTHHozAE1r9boA']

["Best breakfast I've had in a very long time",
 ' very disappointed with breakfast',
 " maybe it's just me",
 " Camellia's was amazing",
 ' had the omelet at the bottom of the menu',
 ' so good',
 ' brother had the Mexican which was tasty',
 ' had a pecan omelet that is to die for',
 ' thinner than most',
 ' enjoyed it',
 ' get your order in asap',
 ' service is personable and awesome.']

In [183]:
import numpy as np 

u_1 = np.array([8, 3]) 
u_2 = np.array([1, -7]) 

U = np.array([u_1, u_2]).T

U

array([[ 8,  1],
       [ 3, -7]])

In [181]:
U @ [5, -3]

array([37, 36])